In [1]:
import robotic as ry
import numpy as np
import time

In [2]:
config = ry.Config()
config.addFile("puzzles/p9-cube-free.g")

In [3]:
print(config.getFrameNames())

['world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'camera_top', 'egoJoint', 'ego', 'goal', 'goal_visible', 'obj', 'goalLarge']


In [4]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"

In [5]:
config.view() # somehow it breaks if I don't call this before setting the camera
time.sleep(1.0)
f = config.getFrame('camera_top')
config.view_setCamera(f)
time.sleep(0.5)

In [6]:
ry.params_clear()
ry.params_add({
    "rrt/stepsize": 0.05,
    "rrt/verbose": 3
})

In [7]:
ATTACHMENT_CONSTANT = 1.02 # to avoid overlapping

q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

goal_to_obj = q_obj - q_goal
goal_to_obj /= np.linalg.norm(goal_to_obj) # now normalized
slope = goal_to_obj[1] / goal_to_obj[0]

ego_radius = config.getFrame(EGO_NAME).getSize()[1]
obj_x, obj_y = map(abs, config.getFrame(OBJ_NAME).getSize()[0:2])

displacement_length = ego_radius + (obj_x / (2 * abs(goal_to_obj[0])) if abs(slope) <= 1 else obj_y / (2 * abs(goal_to_obj[1])))
displacement = goal_to_obj * displacement_length * ATTACHMENT_CONSTANT
q_sub_goal = q_obj + displacement

In [8]:
config.addFrame("point", "floor") \
    .setShape(ry.ST.marker, size=[0.2]) \
    .setRelativePosition(q_sub_goal.tolist() + [0.1])

In [9]:
print("start:", q_start)
print("obj:", q_obj)
print("sub_goal:", q_sub_goal)

start: [0. 0.]
obj: [ 0.7 -1.2]
sub_goal: [ 0.95516541 -1.51063616]


In [10]:
# RRT from start to object
config.view()

rrt = ry.PathFinder()
rrt.setProblem(config, [q_start], [q_sub_goal])
solution = rrt.solve()

path = solution.x
del rrt

  -- rrt success: queries:431 tree sizes: 57 30
  path-length=49


In [13]:
prev_state = q_start
for state in path:
    config.setJointState(state)
    config.view()
    time.sleep(3./path.shape[0]) # animate in 3 seconds
    prev_state = state

**2 code blocks below are for saving rrt path**

In [12]:
config.setJointState(q_start)
config.view()

0

In [36]:
prev_state = q_start
for state in path:
    config.addFrame(str(state), "world") \
        .setShape(ry.ST.sphere, size=[0.02]) \
        .setRelativePosition(state.tolist() + [0.15]) \
        .setColor([0, 0, 0]) \
        .setContact(0)
    prev_state = state

config.view()
config.view_savePng("saves/p9-cube-free-rrt_2")

In [14]:
config.attach(EGO_NAME, OBJ_NAME)

In [15]:
# RRT from object to goal
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_goal = q_goal + displacement

rrt = ry.PathFinder()
rrt.setProblem(config, [q_start], [q_goal])
solution = rrt.solve()
path = solution.x
del rrt

  -- rrt success: queries:550 tree sizes: 70 69
  path-length=89


In [16]:
for state in path:
    config.setJointState(state)
    config.view()
    time.sleep(3./path.shape[0]) # animate in 3 seconds
    prev_state = state

In [18]:
config.view_close()
del config